<a href="https://colab.research.google.com/github/SushovitNanda/SemEval-Food-Hazards/blob/main/PC_Bart_60.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
%%capture
!pip install transformers
!pip install datasets --upgrade
#!pip install torch
import os
import pandas as pd
!pip install evaluate
import evaluate
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import torch
from transformers import (
    BartTokenizer,
    BartForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from datasets import Dataset
import warnings
warnings.filterwarnings("ignore")

# Disable W&B logging
os.environ["WANDB_DISABLED"] = "true"

In [2]:
!wget https://raw.githubusercontent.com/SushovitNanda/SemEval-Food-Hazards/main/Datasets/incidents_train.csv
!wget https://raw.githubusercontent.com/SushovitNanda/SemEval-Food-Hazards/main/Datasets/incidents_val.csv

--2024-12-05 11:37:04--  https://raw.githubusercontent.com/SushovitNanda/SemEval-Food-Hazards/main/Datasets/incidents_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12866710 (12M) [text/plain]
Saving to: ‘incidents_train.csv’

incidents_train.csv 100%[===================>]  12.27M  --.-KB/s    in 0.1s    

2024-12-05 11:37:05 (96.1 MB/s) - ‘incidents_train.csv’ saved [12866710/12866710]

--2024-12-05 11:37:05--  https://raw.githubusercontent.com/SushovitNanda/SemEval-Food-Hazards/main/Datasets/incidents_val.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request se

In [5]:
# Load the dataset
train = pd.read_csv('incidents_train.csv')

# Combine 'title' and 'text' columns to create input data
train['input_text'] = train['title'] + " " + train['text']

# Encode the target labels
label_encoder = LabelEncoder()
train['label'] = label_encoder.fit_transform(train['product-category'])

# Train-test split using stratification
train_df, test_df = train_test_split(
    train[['input_text', 'label']],
    test_size=0.2,
    stratify=train['label'],
    random_state=42
)

# Convert the dataframes into Hugging Face's Dataset format
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Load the BART tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

# Define a tokenization function
def tokenize_function(example):
    return tokenizer(example['input_text'], truncation=True, padding="max_length", max_length=512)

# Tokenize the datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set up a data collator to pad inputs dynamically
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Load the BART model for sequence classification
num_labels = len(label_encoder.classes_)
model = BartForSequenceClassification.from_pretrained("facebook/bart-base", num_labels=num_labels, ignore_mismatched_sizes=True)


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

Map:   0%|          | 0/4065 [00:00<?, ? examples/s]

Map:   0%|          | 0/1017 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [10]:
# Load the F1 metric and specify macro averaging
f1_metric = evaluate.load("f1")

# Define the compute_metrics function to maximize F1 macro average
def compute_metrics(eval_pred):
    logits, labels = eval_pred

    # Extract logits if they are in a tuple
    if isinstance(logits, tuple):
        logits = logits[0]

    # Convert logits to a NumPy array if it isn't already
    logits = np.array(logits)

    # Compute predictions
    predictions = np.argmax(logits, axis=-1)

    # Compute the F1 macro average
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="macro")
    return {"f1": f1["f1"]}

# Update training arguments to focus on F1 score
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=15,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # Use F1 score as the metric for saving best model
    greater_is_better=True       # Ensure higher F1 is considered better
)

# Redefine the Trainer with the updated compute_metrics and early stopping callback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics  # Use F1 macro for evaluation
)

# Add EarlyStoppingCallback to the Trainer with patience of 4 epochs
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2))

# Start training
trainer.train()


Epoch,Training Loss,Validation Loss,F1
1,0.086400,1.597886,0.575895
2,0.669400,1.278304,0.565546
3,0.261600,1.351391,0.594808
4,0.242000,1.424254,0.594571
5,0.179900,1.473776,0.598024
6,0.094000,1.641044,0.591526
7,0.021000,1.632991,0.595408


TrainOutput(global_step=3563, training_loss=0.23683450888848095, metrics={'train_runtime': 3729.2043, 'train_samples_per_second': 16.351, 'train_steps_per_second': 2.047, 'total_flos': 8728130192947200.0, 'train_loss': 0.23683450888848095, 'epoch': 7.0})

In [12]:
# Evaluate the model
predictions = trainer.predict(test_dataset)

# Access the logits/probabilities from the tuple (assuming it's the first element)
preds = predictions.predictions[0].argmax(-1)

# Classification report
target_names = label_encoder.classes_
print(classification_report(test_df['label'], preds, target_names=target_names))

                                                   precision    recall  f1-score   support

                              alcoholic beverages       0.92      0.92      0.92        12
                      cereals and bakery products       0.66      0.85      0.74       134
     cocoa and cocoa preparations, coffee and tea       0.86      0.71      0.78        42
                                    confectionery       0.62      0.68      0.65        34
dietetic foods, food supplements, fortified foods       0.78      0.81      0.79        26
                                    fats and oils       0.67      0.50      0.57         4
                                   feed materials       0.00      0.00      0.00         1
                   food additives and flavourings       0.00      0.00      0.00         2
                           food contact materials       0.00      0.00      0.00         1
                            fruits and vegetables       0.81      0.80      0.81       10